# JNeuron Syntax

JNeuron is really just a library of data structures and methods in the Julia language. Consequently, it has all of the same benefits of working with the Julia language, like speed, easy extensibility to other languages, and a syntax familiar to users of other scripting languages like MATLAB or Python.

Below we outline some common operations that can be performed in JNeuron, and how they can be easily integrated with other tools in Julia, like plotting or computation.

## Loading Packages

Packages are loaded into the Julia environment with the using keyword. Here we are going to load in JNeuron and PyPlot for making figures.

In [ ]:
using JNeuron, PyPlot

## Loading 3d structures

3D imaging reconstructions of neurons can be imported into JNeuron. Right now, Neurolucida (.asc) files are supported, with others soon to come. 

The input function is used to bring microscopic coordinates into JNeuron. The appropriate method is selected based on the file type. 

In [ ]:
#Load 3D Neurolucida file
myimport=input("./data/cell2.asc");

## Manipulating 3D structures

By itself, the data structure of myimport is not terribly useful for simulations. It does, however, provide easy access to the 3D points of the neuron. myimport is made up of "sections," or unbranching parts of the neuron, and each section has a field called "raw" (from Neuron syntax), that contains all of the 3D points of that section. Each column corresponds to a different dimension. Therefore, we could plot the shape of the cell in the xy plane as follows:

In [ ]:
#Plot 3D structure of cell
for i=1:length(myimport.sections)
    plot(myimport.sections[i].raw[:,1],myimport.sections[i].raw[:,2],color="blue")
end

## Creating neurons for simulations

A "Neuron" data structure is the core of JNeuron, and needs to be created from 3D data before running simulations. This is done using the "instantiate" method. The neuron that is created by this method, however, is not very useful: it has no compartments and no ion channels. First, the number of compartments should be determined. The easiest way to do this, is to chose the number of compartments per section based on the length constant of a given section. In JNeuron, this can be done with the set_nsegs! function:

In [ ]:
#Generate instance of neuron type with appropriate sections from 3D data
blank_neuron=instantiate(myimport);

#Create segments based on lambda rule
set_nsegs!(blank_neuron);

## Adding channels to neurons

After a neuron is made from 3D data, and has the appropriate number of compartments, ion channels should be added to the compartments. There are a few methods of doing this. For the same ion channels to be present everywhere, an array of ion channels can be passed to the "add" function along with the neuron data structure from above. A particular ion channel type is constructed with its default constructor, which takes no arguments, such as HH(), or Passive(). 

JNeuron also supports having different ion channel types depending on the region of the cell. There are 4 regions of cells: soma, axon, basal dendrites, and apical dendrites. Therefore a 4 dimensional array of arrays of ion channels can be passed to the add function to make a neuron with different ion channels in those different regions

In [ ]:
#add Hodgkin-Huxley channels to all segments
myneuron1=add(blank_neuron,HH());

#add Hodgkin-Huxley and Passive channels to all segments
myneuron2=add(blank_neuron,(HH(),Passive()));

#add HH and Passive to Soma and Axon, Passive to basal and apical Dendrites
myneuron3=add(blank_neuron,(HH(),Passive()),(HH(),Passive()),Passive(),Passive());

## Adding components to a network

### Neurons

In [ ]:
#Create network with neurons and simulation stop time of 100.0 ms
mynetwork=Network([myneuron1,myneuron2,myneuron3],100.0)

### Extracellular Recording Electrodes

In [ ]:
#Create electrode and add it to the network
electrode=Extracellular([0.0,0.0,0.0]);
add!(mynetwork,electrode);

### Intracellular Recording Electrodes

In [ ]:
#Create intracellular electrode and add it to the network
myintra=Intracellular(1,1,100)
add!(mynetwork,myintra);

### Intracellular Stimulation

In [ ]:
#Create intracellular stimulation and add it to the network
mystim=Stim(2.0,1,1,40,1.0,2.0)
add!(mynetwork,mystim)

## Running a Simulation

In [ ]:
@time run!(mynetwork)